# Data analytics assesment Case
## 1. Identifying The Problem
As part of skills assessment for uber freight i was given a data analytics problem and a data set. The given data is on a route level and its expected to be transformed from one format to another. An example was given. For this report i'll be using Pandas and Numpy just to manipulate data within this notebook, later i'll refine the approach to the solution.

In [1]:
import pandas as pd
import numpy as np

In [2]:
ex = pd.read_excel(
    'Uber Freight_Data Analytics_Assessment 2024.xlsx',
    sheet_name='Example',
    usecols=range(1,16),
    header=2)
inputFormat = ex.loc[[0,1,2]].drop('Unnamed: 15', axis=1)
outputFormat = ex.loc[[7,8,9]]
outputFormat.columns = ex.loc[6]

### Example:
F053978 is one **truckload route outbound**. All loads are picked up in Harrisburg.
When the truck leaves Harrisburg, there are **35,760 pounds** in the truck. The first
delivery is in Barronett (sequence=1). After the truckload leaves from Barronett,
there are **35,666 pounds** in the truckload. Next delivery is Hillsboro where the
truckload leaves with **122 pounds**. Those final pounds will be dropped in the last
delivery: San Geronimo.


In [3]:
inputFormat

,ROUTE ID,SEQ,DATE,MODE,LOAD ID,FLOW TYPE,FROM CITY,FROM ST,FROM ZIP,TO CITY,TO STATE,TO ZIP,DESCRIPTION,LEG WEIGHT
0,F053978,1,2019-01-01 00:00:00,TL,153991,Outbound,Harrisburg,PA,17109,Barronett,WI,54813,Harrisburg -> Barronett,35760
1,F053978,2,2019-01-01 00:00:00,TL,27145,Outbound,Barronett,WI,54813,Hillsboro,OR,97124,Barronett -> Hillsboro,35666
2,F053978,3,2019-01-01 00:00:00,TL,215065,Outbound,Hillsboro,OR,97124,San Geronimo,CA,94963,Hillsboro -> San Geronimo,122


Load 153991 has **94 pounds**, which are picked in Harrisburg and delivered in
Barronett. As it is the **first drop** in the route, it was **picked up last**. Load 27145 has
**35,544 pounds**, which are picked up in Harrisburg and delivered in Hillsboro. Load
215065 has **122 pounds**, which are picked up in Harrisburg and delivered in San
Geronimo. It was picked up first as it was going to be delivered last.

In [4]:
outputFormat

6,LOAD ID,LOAD MODE,ROUTE ID,FLOW TYPE,PICKUP,DROPOFF,ORIGIN CITY,ORIGIN STATE,ORIGIN ZIP,DEST CITY,DEST STATE,DEST ZIP,EARLIESTPICKTIME,LATESTPICKTIME,LOAD WEIGHT
7,153991,LTL,F053978,Outbound,3,4,Harrisburg,PA,17109,Barronett,WI,54813,2019-01-01 00:00:00,2019-01-01 00:00:00,94
8,27145,TL,F053978,Outbound,2,5,Harrisburg,PA,17109,Hillsboro,OR,97124,2019-01-01 00:00:00,2019-01-01 00:00:00,35544
9,215065,LTL,F053978,Outbound,1,6,Harrisburg,PA,17109,San Geronimo,CA,94963,2019-01-01 00:00:00,2019-01-01 00:00:00,122


## 2. Research and Refining  
### Defining the Context:

### Assumptions:
* 0 lb < LTL (Less Than Truckload) <= 20,000 lb
* 20,000 lb < TL (TruckLoad) <= 45,000 lb
* LTL Avg 300 mi/day

### Questions to answer:
#### **1. What is the difference within an inbound route and an outbound route?**

 1. **Inbound Logistics (Inbound Routes)**
    * **Definition:** The movement of **raw materials, components, or finished goods into** a facility from suppliers, manufacturers, or vendors.
    * **Focus:**
      * Receiving goods.
      * Managing supplier relationships.
      * Optimizing transportation costs and lead times.
    * **Key Activities:**
      * Procurement (sourcing materials).
      * Transportation from suppliers to warehouses/factories.
      * Inventory management (staging raw materials for production).
      * Quality checks upon receipt.
    * **Example:**
      * A car factory receives steel, tires, and electronics from suppliers via inbound routes.
      * A retailer’s warehouse receives inventory from manufacturers.

 2. **Outbound Logistics (Outbound Routes)**
  
    * **Definition:** The movement of **finished goods out of** a facility to customers, distributors, or retailers.
    * **Focus:**
      * Delivering products to end-users.
      * Ensuring timely, cost-effective distribution.
      * Managing customer satisfaction.
    * **Key Activities:**
      * Order fulfillment (picking, packing, shipping).
      * Last-mile delivery to customers.
      * Managing carriers (trucks, ships, planes).
      * Reverse logistics (returns).
    * **Example:**
      * An e-commerce warehouse ships orders to customers via outbound routes.
      * A beverage distributor delivers products to grocery stores.
3. **Key Differences**

| Aspect | 	Inbound Logistics | Outbound Logistics |
|---|---|---|  
|Direction | Suppliers → Facility | Facility → Customers/Retailers |
|Stakeholders | Suppliers, procurement teams | Customers, retailers, delivery partners|
|Primary Goal | Ensure materials arrive on time and in full | Ensure products reach customers efficiently |
| Cost Focus | Minimize procurement/transportation costs | Optimize delivery speed and customer service |
| Challenges | Supplier delays, quality control | Last-mile delivery, returns management |
  
   4. **Integration in Supply Chain**

       * **Inbound** ensures raw materials are available for production.
       * **Outbound** ensures finished products reach markets or consumers.
       * Both require coordination with **transportation management systems (TMS)** and **warehouse management systems (WMS).**
  
   5. **Real-World Example**
  
      * **Inbound:** A Walmart distribution center receives pallets of goods from Procter & Gamble.
      * **Outbound:** The same center ships those goods to Walmart stores or directly to online customers.

 * **Why It Matters**

   * **Inbound efficiency** reduces production downtime and inventory costs.
   * **Outbound efficiency** boosts customer satisfaction and brand reputation.
   * Poor inbound logistics can disrupt production; poor outbound logistics can lead to lost sales.
    
#### **2. What is the difference within an LTL and a TL load?**

In logistics and trucking, **LTL (Less Than Truckload)** and **TL (Truckload)** refer to two distinct modes of freight transportation based on shipment size and how space in a truck is utilized. Here’s a breakdown of their differences:
 
 **1. Definitions**
 * **LTL (Less Than Truckload):**
           * Shipments that do not require the full space or weight capacity of a truck.
           * Multiple customers share the same trailer, with each paying for the portion of space their freight occupies.
           * Ideal for smaller shipments (typically **under 10,000 lbs** or **6–12 pallets**).
 * **TL (Truckload):**
            * Shipments large enough to occupy an entire trailer (or close to it).
            * A single customer’s freight fills the truck, and no other cargo is added.
            * Ideal for large shipments (typically over **10,000 lbs** or **10+ pallets**).

 **2. Key Differences**
|**Aspect** | **LTL** | **TL** |
|---|---|---|
|**Shipment Size** | Smaller (partial trailer space) | Large (full trailer) |
|**Cost Structure** | Paid per "space used" + freight class | Flat rate for the entire trailer |
|**Transit Time** | Longer (due to multiple stops for consolidation/deconsolidation) | Faster (direct point-to-point delivery) |
|**Handling** | More handling (transferred between hubs) | Minimal handling (sealed at origin)|
|**Risk of Damage**| Higher (multiple stops and transfers) | Lower (single load, no transfers)|
|**Pricing Factors**| Weight, dimensions, freight class, zones | Distance, fuel, trailer type |
|**Best For** | Small businesses, partial loads, cost-sensitive shipments | Large businesses, urgent/priority shipments, high-value goods|

  **3. When to Use LTL vs. TL**
* **LTL is better if:**
  * Your shipment is **small** (e.g., 1–6 pallets).
  * You want to **save costs** by sharing trailer space.
  * Delivery speed is flexible (e.g., non-urgent goods).
  * You need access to **freight class discounts** (e.g., for lighter, non-dense items).

* **TL is better if:**
  * Your shipment **fills most/all of a trailer.**
  * You need **faster, guaranteed transit times** (e.g., perishable goods).
  * Your cargo is **high-value or fragile** (minimizes handling risk).
  * You require specialized equipment (e.g., refrigerated trailers, flatbeds).

 **4. Practical Examples**
* **LTL:**
    * A small business ships 4 pallets of clothing from Los Angeles to Chicago.
    * A manufacturer sends spare parts to a repair facility.
* **TL:**
    * A furniture company ships 22 pallets of sofas from a warehouse to a retail store.
    * A food distributor transports a full load of frozen goods in a refrigerated trailer.

 **5. Pros and Cons**
|**Mode** |**Pros**|**Cons**|
|---|---|---|
|**LTL** | Cost-effective for small loads, flexible | Slower, higher risk of damage, complex pricing |
|**TL** | Faster, secure, simplified logistics | Expensive for small shipments |

 **6. TL vs. LTL in Special Cases**
 * **Dimensional Weight:** LTL pricing often uses dimensional weight (size vs. actual weight), while TL focuses on actual weight.
 * **Freight Class:** LTL requires assigning a freight class (based on density, stowability, etc.), whereas TL does not.
 * **Accessorials:** LTL may charge extra for liftgates, inside delivery, or residential pickup; TL fees are simpler.

**Which to Choose?**
* **Rule of Thumb:** If your shipment uses **>75% of a trailer’s space/weight**, TL is more efficient. For smaller loads, LTL saves money.
* **Hybrid Option:** Some carriers offer **Partial Truckload (PTL)**, which bridges the gap between LTL and TL for mid-sized shipments.


### My own questions
1. Why are load modes all TL in the input data even when they dont meet the criteria?
   * Since data given is on a route level, all the routes are grouped together meaning that every consolidated route is a truckload. Once the routes are divided, they are calculated to their real mode of transportation.
3. How does delivery sequence translates to load and unload in output?
   * Delivery sequences seems to correspond to a stack structure where the last shipment to be loaded is the first to be delivered. This may be due to how loading and unloading freigths works.
4. How does dates affect earliest pickup times and latest pick times?
   * Due to the inbound flow type, the truck is picking up loads by the sequence shown, assuming a truck drives on average 300 miles per day, earliest and latest pickup times can vary.

#### I asked some questions to the AI to explain more in depth about routes consolidation and shipments
  
### **Outbound Shipments in a Consolidated Route**

* **Earliest Pick Date:**
    * For outbound consolidated shipments (e.g., multiple orders grouped into one truckload), the **earliest possible pick date** is determined by the **latest order** in the batch.
    * Example: If three orders are consolidated, and the earliest order is ready on Monday, but the last order isn’t ready until Wednesday, the **actual pick date** for the entire consolidated load is Wednesday.
    * **Why?** You can’t ship until all orders in the consolidation are ready. The "earliest" possible date is constrained by the slowest order.

* **Scheduling Logic:**
    * All shipments in the consolidated route share the **same pick date**, which is set to the **latest** "ready" date of the grouped orders.
    * Flexibility: If some orders are ready earlier, they must wait in the facility until the consolidated load is finalized.

#### **Inbound Shipments in a Consolidated Route**

* **Pick Date Variability:**
    * Inbound consolidated routes involve collecting goods from **multiple suppliers** (e.g., milk runs). Here, the **earliest and latest pick dates** depend on:
        1. **Supplier availability** (when goods are ready).
        2. **Transit time** between suppliers.
        3. **Carrier scheduling** (e.g., grouping pickups geographically).
    * Example: A truck might pick up from Supplier A (earliest: Monday), then Supplier B (earliest: Tuesday), and Supplier C (earliest: Wednesday). The **latest pick date** for the entire inbound route would be Wednesday.

* **Scheduling Logic:**
    * The carrier must coordinate a route that accommodates all suppliers’ timelines.
    * The **earliest pick date** is the first available pickup time (e.g., Monday), but the **latest pick date** is determined by the last supplier’s availability or transit time between stops.

#### **Key Differences in Consolidated Routes**
| **Aspect** | **Outbound Consolidation** | **Inbound Consolidation** |
|---|---|---|
| **Pick Date Driver** | Determined by the **slowest/latest order** in the batch. | Determined by **supplier availability** and **route efficiency.** |
| **Flexibility** | Facility controls when orders are staged (but must wait for the slowest). | Carrier must adapt to external supplier schedules. |
| **Complexity** | Internally managed (within your facility). | Externally constrained (dependent on suppliers). |
| **Example** | Waiting for 3 customer orders to be packed before a truck can depart. |	A truck visiting 5 suppliers across a region, each with their own ready times.|

#### **Why This Matters**
1. **Outbound:** Efficiency hinges on synchronizing internal workflows to avoid delays (e.g., ensuring all orders in a consolidated load are ready simultaneously).
2. **Inbound:** Requires negotiating with suppliers to align their timelines and minimize idle time in the carrier’s route.

#### **Special Case: Cross-Docking**

In cross-docking, inbound and outbound pick dates may overlap. For example:
* Inbound goods from suppliers are unloaded and immediately reloaded onto outbound trucks.
* Both inbound and outbound pick dates must align perfectly, requiring tight coordination.

#### **TL;DR**
* **Outbound:** All shipments in a consolidated route share the **same pick date**, dictated by the **last order’s readiness.**
* **Inbound:** Pick dates vary by supplier, and the route’s **earliest/latest pick dates** depend on supplier availability and transit time between stops.

This distinction is critical for avoiding delays and optimizing costs in complex supply chains!

### Researching solutions 
#### Load inputData
```Python
inputData = pd.read_excel(
    'Uber Freight_Data Analytics_Assessment 2024.xlsx',
    sheet_name='Input Data',
    usecols=range(0,14),
    header=0)
inputData.sort_values(by='ROUTE ID','SEQ')
```

#### Calculating rough estimate of earliest pickup time
* Cordinates of each city provided by the first part of the assigment can be used to calculate geodesic distance between two places using GeoPy library
* With the rough estimate we can calculate earliest pick up time between each stop
* I should check if all the cities are present for the distance calculations

```Python
from geopy.distance import geodesic
```


Refining solution/PseudoCode
 Breaking up the problem into smaller subproblems
 1. Transform InputData format into OutputData format
      1. Import Input data
      2. Obtain every Route ID
      3. Obtain Cities coordinates
      4. Obtain Output data headers
      5. Loop through every Route
      6. Calculate each column
         * outLoadId = inLoadId
         * if loadWeigth > 20,000:
               loadMode = "TL"
           else:
               loadMode = "LTL"
         * routeId = routeId
         * Pickup = range(maxSeq,0,-1)[seq - 1]
         * Dropoff = maxSeq + seq
         * originCity = inputData[inputData['ROUTE ID'] == routeId and inputData['SEQ'] == 1].loc['FROM CITY']
         * originState = inputData[inputData['ROUTE ID'] == routeId and inputData['SEQ'] == 1].loc['FROM ST']
         * originZip = inputData[inputData['ROUTE ID'] == routeId and inputData['SEQ'] == 1].loc['FROM ZIP']
         * destCity = "TO CITY"
         * destState = "TO STATE"
         * destZip = "TO ZIP"
         * earliestPick
         * if geoDist > 300miles:
               earliestPick = seq1Date + (geoDist // 300)
         * lastestPick = 'DATE'
         * loadWeight = loadweightsList[seq]
         * for n in range(len(inputData['LEG WEIGHT']) - 1):
               remind -= legWeight[n + 1]
               loadWeightsList[n] = remind
               loadWeightsList[-1] = remind
      7. Assign each value
      8. Testing
      9. Import Data to excel

In [1]:
'''
Refining solution/PseudoCode
 Breaking up the problem into smaller subproblems
 1. Transform InputData format into OutputData format
      1. Import Input data
      2. Obtain every Route ID
      3. Obtain Cities coordinates
      4. Obtain Output data headers
      5. Loop through every Route
      6. Calculate each column
         * outLoadId = inLoadId
         * if loadWeigth > 20,000:
               loadMode = "TL"
           else:
               loadMode = "LTL"
         * routeId = routeId
         * Pickup = range(maxSeq,0,-1)[seq - 1]
         * Dropoff = maxSeq + seq
         * originCity = inputData[inputData['ROUTE ID'] == routeId and inputData['SEQ'] == 1].loc['FROM CITY']
         * originState = inputData[inputData['ROUTE ID'] == routeId and inputData['SEQ'] == 1].loc['FROM ST']
         * originZip = inputData[inputData['ROUTE ID'] == routeId and inputData['SEQ'] == 1].loc['FROM ZIP']
         * destCity = "TO CITY"
         * destState = "TO STATE"
         * destZip = "TO ZIP"
         * earliestPick
         * if geoDist > 300miles:
               earliestPick = seq1Date + (geoDist // 300)
         * lastestPick = 'DATE'
         * loadWeight = loadweightsList[seq]
         * for n in range(len(inputData['LEG WEIGHT']) - 1):
               remind -= legWeight[n + 1]
               loadWeightsList[n] = remind
               loadWeightsList[-1] = remind
      7. Assign each value
      8. Testing
      9. Import Data to excel
'''

'\nRefining solution/PseudoCode\n Breaking up the problem into smaller subproblems\n 1. Transform InputData format into OutputData format\n      1. Import Input data\n      2. Obtain every Route ID\n      3. Obtain Cities coordinates\n      4. Obtain Output data headers\n      5. Loop through every Route\n      6. Calculate each column\n         * outLoadId = inLoadId\n         * if loadWeigth > 20,000:\n               loadMode = "TL"\n           else:\n               loadMode = "LTL"\n         * routeId = routeId\n         * Pickup = range(maxSeq,0,-1)[seq - 1]\n         * Dropoff = maxSeq + seq\n         * originCity = inputData[inputData[\'ROUTE ID\'] == routeId and inputData[\'SEQ\'] == 1].loc[\'FROM CITY\']\n         * originState = inputData[inputData[\'ROUTE ID\'] == routeId and inputData[\'SEQ\'] == 1].loc[\'FROM ST\']\n         * originZip = inputData[inputData[\'ROUTE ID\'] == routeId and inputData[\'SEQ\'] == 1].loc[\'FROM ZIP\']\n         * destCity = "TO CITY"\n         * 

In [5]:
# Loading inputData
inputData = pd.read_excel(
    'Uber Freight_Data Analytics_Assessment 2024.xlsx',
    sheet_name='Input Data',
    usecols=range(0,14),
    header=0)
inputData.sort_values(by=['ROUTE ID','SEQ'])

# Initializing outputData dataframe
outputData = pd.read_excel(
    'Uber Freight_Data Analytics_Assessment 2024.xlsx',
    sheet_name='Output template',
    usecols=range(0,15))

"""
I decided that importing a bigger database is a better aproach since USA has more than 10,000 cities
still, 140 cities from the inputData were missing from the imported database
"""
# Cities Coordinates
citiesCoor = pd.read_excel(
    'Uber Freight Engineering - Customer Demands Business Case.xlsx',
    sheet_name='Data',
    usecols=[1,2,4,5],
    header=0)

# Checking for missing cities
inputCities = set(inputData['FROM CITY']) | set(inputData['TO CITY'])
# inputCities 971
# citiesCoor 370

# cities coordinates and inputData Cities are different
missingCities = set([name.upper() for name in inputCities]) - set(citiesCoor['CUSTOMER CITY'])

# missingCities 905

# Importing cities from US cities database
uscities = pd.read_csv('uscities.csv', usecols = [0,2,6,7])

# Making set of all found cities
foundCities = inputCities & set(uscities['city'])
# Cities not found in the database totals 140

# Adding coordinates
mask = uscities['city'].isin(foundCities)
cities = uscities[mask]

In [6]:
from geopy.distance import geodesic
from datetime import timedelta

#Assuming LTL carriers cover 300mi/day
AVGMILEAGE = 300


def loadWeight(index):
    """
    Calculates the load weight of each trip.

    Uses index of input dataframe.
    """
    route = inputData.loc[index]['ROUTE ID']
    flow = inputData.loc[index]['FLOW TYPE']
    seq = inputData.loc[index]['SEQ']
    seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == route])


    if flow == 'Outbound':
        if seq == len(seqList):
            return inputData.loc[index]['LEG WEIGHT']
        nextWeight = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seq + 1)]['LEG WEIGHT']
        nextWeight = nextWeight.iloc[0]

        return inputData.loc[index]['LEG WEIGHT'] - nextWeight
        
    else:
        if seq == 1:
            return inputData.loc[index]['LEG WEIGHT']
        prevWeight = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seq - 1)]['LEG WEIGHT']
        prevWeight = prevWeight.iloc[0]

        return inputData.loc[index]['LEG WEIGHT'] - prevWeight

def pickDrop(index):
    """
    Determines de pickup and dropoff of each trip.
    """
    route = inputData.loc[index]['ROUTE ID']
    flow = inputData.loc[index]['FLOW TYPE']
    seq = inputData.loc[index]['SEQ']
    seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == route])
    seqList.sort()

    if flow == 'Outbound':
        pickup = seqList[len(seqList) - seq]
        dropoff = seq + max(seqList)
    else:
        pickup = seq
        dropoff = seqList[len(seqList) - seq] + max(seqList)

    return pickup, dropoff

    

def dates(index):
    """
    Determines and calculates the earliest and latest pickup dates.
    Uses geodesic calculations to determine a rough estimate distance for each stop,
    
    In Inbound routes the earliest pickup depends on the supplier, 
    and the latest on the calculated time of arrival

    In Outbound routes the earliest and latest pickup depends on the last load in the sequence.
    """
    route = inputData.loc[index]['ROUTE ID']
    flow = inputData.loc[index]['FLOW TYPE']
    seq = inputData.loc[index]['SEQ']
    seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == route])
    seqList.sort()

    if flow == 'Outbound':
        eDate = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seqList[-1])]['DATE'].iloc[0]
        lDate = eDate

        return eDate, lDate
    else:
        eDate = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seq)]['DATE'].iloc[0]
        
        fromCity = inputData.iloc[index]['FROM CITY']
        fromSt = inputData.iloc[index]['FROM ST']
        toCity = inputData.iloc[index]['TO CITY']
        toSt = inputData.iloc[index]['TO STATE']

        # Checks that the origin and destination cities exists whitin the data base to perform the calculations
        check1 = cities[(cities['city'] == fromCity) & (cities['state_id'] == fromSt)].any()
        check2 = cities[(cities['city'] == toCity) & (cities['state_id'] == toSt)].any()

        
        if check1.any() and check2.any():
            fromCityData = cities.loc[(cities['city'] == fromCity) & (cities['state_id'] == fromSt)]
            toCityData = cities.loc[(cities['city'] == toCity) & (cities['state_id'] == toSt)]

            
            # Geodesic calculation since it takes into account earth curvature
            estDist = geodesic((fromCityData['lat'].item(),fromCityData['lng'].item()),
                               (toCityData['lat'].item(),toCityData['lng'].item())).mi
                
            sumDays = int(estDist // AVGMILEAGE)

            lDate = eDate + timedelta(days=sumDays)
            
            return eDate, lDate

        #If the cities wheren't found, it defaults into the original earliest Pickup
        lDate = eDate

        # Storing cities not found
        if check1.any():
            citiesNt.append((fromCity,fromSt))
        if check2.any():
            citiesNt.append((toCity,toSt))
        
        return eDate, lDate
        
# Lists initialization
# whole List assigments are faster than dataframe row iterations
loadWeightLs = []
loadModeLs = []
pickup = []
dropoff = []
originCity = []
originState = []
originZip = []
destCity = []
destState = []
destZip = []
earliestDateLs = []
latestDateLs = []

#Cities not found
citiesNt = []

for index, row in inputData.iterrows():

    #Load weight
    loadWeightLs.append(loadWeight(index))

    # Loadmode
    if loadWeightLs[index] > 20000:
        loadModeLs.append('TL')
    else:
        loadModeLs.append('LTL')

    #Pick and Drop sequence
    pick, drop = pickDrop(index)
    pickup.append(pick)
    dropoff.append(drop)

    # Origin city
    if row['FLOW TYPE'] == 'Outbound':
        originCity.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == 1)]['FROM CITY'].iloc[0])
        originState.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == 1)]['FROM ST'].iloc[0])
        originZip.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == 1)]['FROM ZIP'].iloc[0])
    else:
        seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == row['ROUTE ID']])
        originCity.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == seqList[-1])]['FROM CITY'].iloc[0])
        originState.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == seqList[-1])]['FROM ST'].iloc[0])
        originZip.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == seqList[-1])]['FROM ZIP'].iloc[0])

    #Destiny city
    destCity.append(row['TO CITY'])
    destState.append(row['TO STATE'])
    destZip.append(row['TO ZIP'])

    #Earliest and Latest pickup dates
    earliestDate, latestDate = dates(index)
    earliestDateLs.append(earliestDate)
    latestDateLs.append(latestDate)


# Populating outputData
outputData['LOAD ID'] = inputData['LOAD ID']
outputData['ROUTE ID'] = inputData['ROUTE ID']
outputData['FLOW TYPE'] = inputData['FLOW TYPE']
outputData['LOAD WEIGHT'] = loadWeightLs
outputData['LOAD MODE'] = loadModeLs
outputData['PICKUP'] = pickup
outputData['DROPOFF'] = dropoff
outputData['ORIGIN CITY'] = originCity
outputData['ORIGIN STATE'] = originState
outputData['ORIGIN ZIP'] = originZip
outputData['DEST CITY'] = destCity
outputData['DEST STATE'] = destState
outputData['DEST ZIP'] = destZip
outputData['EARLIESTPICKTIME'] = earliestDateLs
outputData['LATESTPICKTIME'] = latestDateLs


In [7]:
outputData

,LOAD ID,LOAD MODE,ROUTE ID,FLOW TYPE,PICKUP,DROPOFF,ORIGIN CITY,ORIGIN STATE,ORIGIN ZIP,DEST CITY,DEST STATE,DEST ZIP,EARLIESTPICKTIME,LATESTPICKTIME,LOAD WEIGHT
0,90637,TL,F000570,Inbound,1,4,Cedar Grove,WV,25039,Cedar Grove,WV,25039,2019-04-08,2019-04-08,33547
1,4769,LTL,F000570,Inbound,2,3,Cedar Grove,WV,25039,Harrisburg,PA,17109,2019-04-09,2019-04-09,5286
2,5310,LTL,F001155,Inbound,1,4,Point Arena,CA,95468,Point Arena,CA,95468,2019-05-06,2019-05-07,15748
3,91180,TL,F001155,Inbound,2,3,Point Arena,CA,95468,Harrisburg,PA,17109,2019-05-07,2019-05-15,26531
4,116780,TL,F001722,Inbound,1,4,Dewey,OK,74029,Dewey,OK,74029,2019-06-06,2019-06-06,28647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,181062,LTL,F143744,Outbound,1,6,Carlsbad,CA,92010,Grove,OK,74344,2019-09-20,2019-09-20,3503
1284,211343,LTL,F143875,Outbound,4,5,Carlsbad,CA,92010,Oklahoma City,OK,73173,2019-09-27,2019-09-27,2118
1285,210038,LTL,F143875,Outbound,3,6,Carlsbad,CA,92010,East Bernstadt,KY,40729,2019-09-27,2019-09-27,4163
1286,50306,TL,F143875,Outbound,2,7,Carlsbad,CA,92010,Washington,VA,22747,2019-09-27,2019-09-27,37204


#### Obtaining missing cities coordinates

In [56]:
# Find missing by merge indicator
input_cities = inputData.iloc[:, [6, 7]].rename(columns={'FROM CITY': 'City', 'FROM ST': 'State'})
valid_cities = cities.iloc[:, [0, 1]].rename(columns={'city': 'City', 'state_id': 'State'})

merged = input_cities.merge(
    valid_cities.drop_duplicates(),  # Avoid duplicate matches
    on=['City', 'State'],
    how='left',
    indicator=True
)

mask = merged['_merge'] == 'left_only'
rows_with_missing = inputData[mask]

missing_combinations = merged.loc[mask, ['City', 'State']].drop_duplicates()

print("Rows with missing city-state combinations:")
print(rows_with_missing.iloc[:,[0,1,4,6,7,9,10]])

print("\nUnique missing city-state pairs:")
print(missing_combinations)

Rows with missing city-state combinations:
     ROUTE ID  SEQ  LOAD ID          FROM CITY FROM ST      TO CITY TO STATE
0     F000570    1    90637            Rickman      TN  Cedar Grove       WV
13    F004286    2     4918          Pall Mall      TN   Harrisburg       PA
22    F007131    2     6356      Sugarloaf Key      FL    Asheville       NC
30    F009044    2     5910               Totz      KY    Asheville       NC
33    F010435    1    91312          Dover AFB      DE    Highfalls       NC
...       ...  ...      ...                ...     ...          ...      ...
1238  F140966    3    24883  Greenwell Springs      LA    Melbourne       FL
1263  F142775    3   150750          Blakeslee      PA      Bayside       NY
1264  F142775    4   209475            Bayside      NY  Bishopville       MD
1268  F142900    4   212355           Redfield      NY     Westford       NY
1283  F143744    3   181062            Meriden      WY        Grove       OK

[178 rows x 7 columns]

Unique m

In [55]:
from geopy.geocoders import Nominatim

# Instatiate a new Nominatim client
app = Nominatim(user_agent="MyApp")
import time

citiesNtCoor = []

# Query cities not found
counter = 0
for index, row in missing_combinations.iterrows():
    counter += 1
    print(counter, end='\r')
    citiesNtCoor.append(app.geocode(f"{row[0]} {row[1]}"))
    time.sleep(1.1) # Mandatory 1s delay between requests

1

C:\Users\Edgar Tijerina\AppData\Local\Temp\ipykernel_1208\2491546372.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  citiesNtCoor.append(app.geocode(f"{row[0]} {row[1]}"))


2

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Pall+Mall+TN&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [33]:
# Set new cities found
foundCities = {'city':[], 'state_id':[], 'lat':[], 'lng':[]}
citiesRemaning = {'city':[],'state_id':[]}
for index, city in enumerate(citiesNtCoor):
    try:
        foundCities['lat'].append(city.latitude)
        foundCities['lng'].append(city.longitude)    
        foundCities['city'].append(missing_combinations.iloc[index,0])
        foundCities['state_id'].append(missing_combinations.iloc[index,1])
    except AttributeError:
        citiesRemaning['city'].append(missing_combinations.iloc[index,0])
        citiesRemaning['state_id'].append(missing_combinations.iloc[index,1])

print('Remaning cities not found: ', citiesRemaning) 

# Update cities dataframe and drop duplicates
fndCities = pd.DataFrame(foundCities)
citiesFull = pd.concat([cities,fndCities], ignore_index = True)
citiesFull.drop_duplicates(subset=['city','state_id'], keep='first', inplace=True)

Remaning cities not found:  {'city': [], 'state_id': []}


#### **Testing**
To validate the data obtained some tests were developed. The tests consist of 3 randomly selected sample Routes compared to their manually solved counterparts.


In [34]:
# Importing samples
sampleData = pd.read_excel(
    'Uber Freight_Data Analytics_Assessment 2024.xlsx',
    sheet_name='Example',
    usecols=range(1,16),
    header=0,
    skiprows=39)

# Importing sample trip distance
sampleDist = pd.read_excel(
    'Uber Freight_Data Analytics_Assessment 2024.xlsx',
    sheet_name='Example',
    usecols=[16],
    header=0,
    skiprows=39,
    ).dropna()

In [35]:
sample1 = sampleData.iloc[0:3]
sample2 = sampleData.iloc[3:7]
sample3 = sampleData.iloc[7:9]

In [36]:
#Checking for differences
def test1():
    index = inputData[inputData['ROUTE ID'] == sample1['ROUTE ID'].iloc[0]].index
    
    print('Test1:\n', 'OutputData:\n', outputData.iloc[index],'\nSample:\n', sample1)

    # Trip time estimate
    dists = {'TRIP DIST':[]}
    sumDays = []
    for i in index[:-1]:
        
        fromCity = inputData.iloc[i]['FROM CITY']
        fromSt = inputData.iloc[i]['FROM ST']
        toCity = inputData.iloc[i]['TO CITY']
        toSt = inputData.iloc[i]['TO STATE']

        fromCityData = citiesFull.loc[(citiesFull['city'] == fromCity) & (citiesFull['state_id'] == fromSt)]
        toCityData = citiesFull.loc[(citiesFull['city'] == toCity) & (citiesFull['state_id'] == toSt)]
    
        dists['TRIP DIST'].append(geodesic((fromCityData['lat'].item(),fromCityData['lng'].item()),
                                   (toCityData['lat'].item(),toCityData['lng'].item())).mi)
    
    distsDf = pd.DataFrame(dists)
    print('\n Sample Distances: \n',sampleDist,'\n', 'Estimated Distances: \n', distsDf,'\n')
    
def test2():
    print('Test2:\n', 'OutputData:\n', outputData[outputData['ROUTE ID'] == sample2['ROUTE ID'].iloc[0]],'\nSample:\n', sample2)

def test3():
    print('Test3:\n', 'OutputData:\n', outputData[outputData['ROUTE ID'] == sample3['ROUTE ID'].iloc[0]],'\nSample:\n', sample3)

test1()
test2()
test3()

Test1:
 OutputData:
      LOAD ID LOAD MODE ROUTE ID FLOW TYPE  PICKUP  DROPOFF ORIGIN CITY  \
160    11751       LTL  F048342   Inbound       1        6      Exmore   
161   110149        TL  F048342   Inbound       2        5        Knox   
162    11232       LTL  F048342   Inbound       3        4    Bellview   

    ORIGIN STATE  ORIGIN ZIP DEST CITY DEST STATE  DEST ZIP EARLIESTPICKTIME  \
160           VA       23350  Carlsbad         CA     92010       2019-01-08   
161           PA       16232  Carlsbad         CA     92010       2019-01-09   
162           IL       61604  Carlsbad         CA     92010       2019-01-11   

    LATESTPICKTIME  LOAD WEIGHT  
160     2019-01-09         6070  
161     2019-01-09        25291  
162     2019-01-11         3994   
Sample:
    LOAD ID LOAD MODE ROUTE ID FLOW TYPE  PICKUP  DROPOFF ORIGIN CITY  \
0    11751       LTL  F048342   Inbound       1        6      Exmore   
1   110149        TL  F048342   Inbound       2        5        Knox   

ValueError: can only convert an array of size 1 to a Python scalar

**Test 1:**
We can see that Origin City and Destiny city is incorrect and that our trip distance is inaccurate. corrections to the code is needed 

**Test 2 and Test 3:**
Test 2 and 3 passed the test succesfully, no need for corrections needed.

#### **Corrected code**

In [45]:
from geopy.distance import geodesic
from datetime import timedelta

#Assuming LTL carriers cover 300mi/day
AVGMILEAGE = 300


def loadWeight(index):
    """
    Calculates the load weight of each trip.

    Uses index of input dataframe.
    """
    route = inputData.loc[index]['ROUTE ID']
    flow = inputData.loc[index]['FLOW TYPE']
    seq = inputData.loc[index]['SEQ']
    seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == route])


    if flow == 'Outbound':
        if seq == len(seqList):
            return inputData.loc[index]['LEG WEIGHT']
        nextWeight = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seq + 1)]['LEG WEIGHT']
        nextWeight = nextWeight.iloc[0]

        return inputData.loc[index]['LEG WEIGHT'] - nextWeight
        
    else:
        if seq == 1:
            return inputData.loc[index]['LEG WEIGHT']
        prevWeight = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seq - 1)]['LEG WEIGHT']
        prevWeight = prevWeight.iloc[0]

        return inputData.loc[index]['LEG WEIGHT'] - prevWeight

def pickDrop(index):
    """
    Determines de pickup and dropoff of each trip.
    """
    route = inputData.loc[index]['ROUTE ID']
    flow = inputData.loc[index]['FLOW TYPE']
    seq = inputData.loc[index]['SEQ']
    seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == route])
    seqList.sort()

    if flow == 'Outbound':
        pickup = seqList[len(seqList) - seq]
        dropoff = seq + max(seqList)
    else:
        pickup = seq
        dropoff = seqList[len(seqList) - seq] + max(seqList)

    return pickup, dropoff

    

def dates(index):
    """
    Determines and calculates the earliest and latest pickup dates.
    Uses geodesic calculations to determine a rough estimate distance for each stop,
    
    In Inbound routes the earliest pickup depends on the supplier, 
    and the latest on the calculated time of arrival

    In Outbound routes the earliest and latest pickup depends on the last load in the sequence.
    """
    route = inputData.loc[index]['ROUTE ID']
    flow = inputData.loc[index]['FLOW TYPE']
    seq = inputData.loc[index]['SEQ']
    seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == route])
    seqList.sort()

    if flow == 'Outbound':
        eDate = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seqList[-1])]['DATE'].iloc[0]
        lDate = eDate

        return eDate, lDate
    else:
        eDate = inputData.loc[(inputData['ROUTE ID'] == route ) & (inputData['SEQ'] == seq)]['DATE'].iloc[0]
        
        fromCity = inputData.iloc[index]['FROM CITY']
        fromSt = inputData.iloc[index]['FROM ST']
        toCity = inputData.iloc[index]['TO CITY']
        toSt = inputData.iloc[index]['TO STATE']

        # Checks that the origin and destination cities exists whitin the data base to perform the calculations
        check1 = cities[(cities['city'] == fromCity) & (cities['state_id'] == fromSt)].any()
        check2 = cities[(cities['city'] == toCity) & (cities['state_id'] == toSt)].any()

        
        if check1.any() and check2.any():
            fromCityData = cities.loc[(cities['city'] == fromCity) & (cities['state_id'] == fromSt)]
            toCityData = cities.loc[(cities['city'] == toCity) & (cities['state_id'] == toSt)]

            
            # Geodesic calculation since it takes into account earth curvature
            estDist = geodesic((fromCityData['lat'].item(),fromCityData['lng'].item()),
                               (toCityData['lat'].item(),toCityData['lng'].item())).mi
                
            sumDays = int(estDist // AVGMILEAGE)

            lDate = eDate + timedelta(days=sumDays)
            
            return eDate, lDate

        #If the cities wheren't found, it defaults into the original earliest Pickup
        lDate = eDate

        # Storing cities not found
        if check1.any():
            citiesNt.append((fromCity,fromSt))
        if check2.any():
            citiesNt.append((toCity,toSt))
        
        return eDate, lDate
        
# Lists initialization
# whole List assigments are faster than dataframe row iterations
loadWeightLs = []
loadModeLs = []
pickup = []
dropoff = []
originCity = []
originState = []
originZip = []
destCity = []
destState = []
destZip = []
earliestDateLs = []
latestDateLs = []

#Cities not found
citiesNt = []

for index, row in inputData.iterrows():

    #Load weight
    loadWeightLs.append(loadWeight(index))

    # Loadmode
    if loadWeightLs[index] > 20000:
        loadModeLs.append('TL')
    else:
        loadModeLs.append('LTL')

    #Pick and Drop sequence
    pick, drop = pickDrop(index)
    pickup.append(pick)
    dropoff.append(drop)

    # Origin city and Destiny city
    if row['FLOW TYPE'] == 'Outbound':

        # Origin city is the last city where the package is shipped off
        originCity.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == 1)]['FROM CITY'].iloc[0])
        originState.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == 1)]['FROM ST'].iloc[0])
        originZip.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == 1)]['FROM ZIP'].iloc[0])
        
        # Destiny city is where the package is droped off
        destCity.append(row['TO CITY'])
        destState.append(row['TO STATE'])
        destZip.append(row['TO ZIP'])
    
    else:
        # Inbound route
        seqList = list(inputData['SEQ'].loc[inputData['ROUTE ID'] == row['ROUTE ID']])
        
        # Destiny city is the last city in the trip
        destCity.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == len(seqList))]['TO CITY'].iloc[0])
        destState.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == len(seqList))]['TO STATE'].iloc[0])
        destZip.append(inputData.loc[(inputData['ROUTE ID'] == row['ROUTE ID']) & (inputData['SEQ'] == len(seqList))]['TO ZIP'].iloc[0])
        
        # Origin city is the city where the load is picked up
        originCity.append(row['FROM CITY'])
        originState.append(row['FROM ST'])
        originZip.append(row['FROM ZIP'])

    #Earliest and Latest pickup dates
    earliestDate, latestDate = dates(index)
    earliestDateLs.append(earliestDate)
    latestDateLs.append(latestDate)


# Populating outputData
outputData['LOAD ID'] = inputData['LOAD ID']
outputData['ROUTE ID'] = inputData['ROUTE ID']
outputData['FLOW TYPE'] = inputData['FLOW TYPE']
outputData['LOAD WEIGHT'] = loadWeightLs
outputData['LOAD MODE'] = loadModeLs
outputData['PICKUP'] = pickup
outputData['DROPOFF'] = dropoff
outputData['ORIGIN CITY'] = originCity
outputData['ORIGIN STATE'] = originState
outputData['ORIGIN ZIP'] = originZip
outputData['DEST CITY'] = destCity
outputData['DEST STATE'] = destState
outputData['DEST ZIP'] = destZip
outputData['EARLIESTPICKTIME'] = earliestDateLs
outputData['LATESTPICKTIME'] = latestDateLs

In [54]:
#Checking for differences
def test1():
    index = inputData[inputData['ROUTE ID'] == sample1['ROUTE ID'].iloc[0]].index
    
    print('Test1:\n', 'OutputData:\n', outputData.iloc[index],'\nSample:\n', sample1)

    # Trip time estimate
    dists = {'TRIP DIST':[]}
    sumDays = []
    
    # If the cities missing weren't searched, it might fail to test the last distance. Adjust index[:-1], index[:1]
    for i in index[:1]:
        
        fromCity = inputData.iloc[i]['FROM CITY']
        fromSt = inputData.iloc[i]['FROM ST']
        toCity = inputData.iloc[i]['TO CITY']
        toSt = inputData.iloc[i]['TO STATE']

        fromCityData = citiesFull.loc[(citiesFull['city'] == fromCity) & (citiesFull['state_id'] == fromSt)]
        toCityData = citiesFull.loc[(citiesFull['city'] == toCity) & (citiesFull['state_id'] == toSt)]

        from_lat = fromCityData['lat'].iloc[0]
        from_lng = fromCityData['lng'].iloc[0]
        to_lat = toCityData['lat'].iloc[0]
        to_lng = toCityData['lng'].iloc[0]

        dists['TRIP DIST'].append(geodesic((from_lat, from_lng),
                                   (to_lat, to_lng)).mi)
    
    distsDf = pd.DataFrame(dists)
    print('\n Sample Distances: \n',sampleDist,'\n', 'Estimated Distances: \n', distsDf,'\n')
    accu = round(((distsDf.mean()/sampleDist.mean())*100).item() - 100)
    print('Trip distance accuracy:', accu,'%\n')
    
def test2():
    print('Test2:\n', 'OutputData:\n', outputData[outputData['ROUTE ID'] == sample2['ROUTE ID'].iloc[0]],'\nSample:\n', sample2)

def test3():
    print('Test3:\n', 'OutputData:\n', outputData[outputData['ROUTE ID'] == sample3['ROUTE ID'].iloc[0]],'\nSample:\n', sample3)

test1()
test2()
test3()

print('\nCities not found:', len(citiesNt))

Test1:
 OutputData:
      LOAD ID LOAD MODE ROUTE ID FLOW TYPE  PICKUP  DROPOFF ORIGIN CITY  \
160    11751       LTL  F048342   Inbound       1        6      Exmore   
161   110149        TL  F048342   Inbound       2        5        Knox   
162    11232       LTL  F048342   Inbound       3        4    Bellview   

    ORIGIN STATE  ORIGIN ZIP DEST CITY DEST STATE  DEST ZIP EARLIESTPICKTIME  \
160           VA       23350  Carlsbad         CA     92010       2019-01-08   
161           PA       16232  Carlsbad         CA     92010       2019-01-09   
162           IL       61604  Carlsbad         CA     92010       2019-01-11   

    LATESTPICKTIME  LOAD WEIGHT  
160     2019-01-09         6070  
161     2019-01-09        25291  
162     2019-01-11         3994   
Sample:
    LOAD ID LOAD MODE ROUTE ID FLOW TYPE  PICKUP  DROPOFF ORIGIN CITY  \
0    11751       LTL  F048342   Inbound       1        6      Exmore   
1   110149        TL  F048342   Inbound       2        5        Knox   

### **Results**

With the code corrected the output now has the proper format, with the caveat that the latest pickup date for inbound routes is underestimated due to the limitation of being only geodesic calculations and not taking into account real USA routes. This could be improved in the future with better data or tweaks to the code.

In [ ]:
outputData.to_csv('outputData.csv', index=False)

In [ ]:
outputData